**Task 04: Graph querying**

In [1]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2024-2025/master/Assignment4/course_materials"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.7 MB/s eta 0:00:00


In [2]:
from rdflib import Graph, Namespace, Literal
g = Graph()
g.parse(github_storage+"/rdf/example3.rdf", format="xml")

<Graph identifier=Nd1df2bea2c804001baf37d5dd15f7b93 (<class 'rdflib.graph.Graph'>)>

Listamos todos los recursos que contienen la propiedad VCARD:FN

In [3]:
from rdflib.plugins.sparql import prepareQuery


VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")


q1 = prepareQuery('''
  SELECT ?Subject ?FullName WHERE {
    ?Subject vcard:FN ?FullName.
  }
  ''',
  initNs = { "vcard": VCARD}
)


for r in g.query(q1):
  print(r.Subject)
  print(r.FullName)

http://somewhere#JaneSmith
Jane Smith
http://somewhere#JohnSmith
John Smith
http://somewhere#SarahJones
Sarah Jones
http://somewhere#MattJones
Matt Jones


Repetimos la anterior consulta, pero pidiendo ahora además los nombres completos de los sujetos

In [4]:
q2 = prepareQuery('''
  SELECT ?Subject ?FullName WHERE {
    ?Subject vcard:FN ?FullName.
  }
  ''',
  initNs = { "vcard": VCARD}
)

for r in g.query(q2):
  print(r.Subject, r.FullName)

http://somewhere#JaneSmith Jane Smith
http://somewhere#JohnSmith John Smith
http://somewhere#SarahJones Sarah Jones
http://somewhere#MattJones Matt Jones


Obtenemos todos los recursos que contienen "Smith" como nombre de familia

In [5]:
from rdflib import XSD

q3 = prepareQuery('''
  SELECT ?Subject WHERE {
    ?Subject vcard:Family ?Family.
  }
  ''',
  initNs = { "vcard": VCARD}
)

for r in g.query(q3, initBindings = {'?Family' : Literal('Smith', datatype=XSD.string)}): #Equivalente a "Smith"^^xsd:string
  print(r.Subject)

http://somewhere#JaneSmith
http://somewhere#JohnSmith


Obtenemos todos los elementos que contienen un email asociado

In [6]:
from rdflib import FOAF

q4 = prepareQuery('''
  SELECT ?Subject ?Email WHERE {
    ?Subject foaf:email ?Email.
  }
  ''',
  initNs = { "foaf": FOAF}
)

for r in g.query(q4):
  print(r.Subject,r.Email)

http://somewhere#JaneSmith jSmith@somewhere.com
http://somewhere#SarahJones sJones@somewhere.com


Consultamos todos los que conocen (FOAF:knows) a "Jane Smith" y obtenemos sus nombres de pila (VCARD:Given)

In [7]:
q5 = prepareQuery('''
  SELECT  ?Subject ?Given  WHERE {
    ?Subject foaf:knows ?JaneSmith.
	?JaneSmith vcard:FN ?JaneSmithFullName.
	?Subject vcard:Given ?Given.
  }
  ''',
  initNs = { "foaf": FOAF, "vcard": VCARD, "xsd":XSD}
)

for r in g.query(q5, initBindings = {'?JaneSmithFullName' : Literal('Jane Smith', datatype=XSD.string)}):
  print(r.Subject, r.Given)

http://somewhere#JohnSmith John
http://somewhere#MattJones Matt
